<a href="https://colab.research.google.com/github/edwardb1203/COMP590_NLP/blob/main/Copy_of_COMP590_158_Hw1_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get the data ready

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# See what is in your google drive and make sure the files are in there
!ls "/content/drive/My Drive"

'2020 hackathon submission.gdoc'
'301 Common Issues.gdoc'
'311 Review Doc.gdoc'
'50 States Problem Set.gdoc'
'58360511041__87EAE5F4-502A-41B2-8F05-6CDE955AEA31 (1).JPG'
 58360511041__87EAE5F4-502A-41B2-8F05-6CDE955AEA31.JPG
 59241893359__A777C512-FE99-4459-94A7-D39BEB3D2167.JPG
 59373746294__79322360-FD3B-4571-922E-814572327C3A.JPG
 59958850954__C978A09A-FB2D-4E4B-BF63-C88E9B0B85EE.JPG
 60099575199__51C8B0A0-8B71-4D4A-A84A-538F9E11CB42.JPG
 61032650137__68D1DAA8-EDF8-4E4E-BE72-2C7A7C9189EC.JPG
 62249157674__06CEB47F-A2BC-4B67-BEFE-1F1DEE244F16.JPG
 62256442322__4658B34A-7157-4ACB-B136-5BEBDE68E2E7.JPG
 63855138430__0824DA8F-CD85-4582-B8B8-39303A185707.HEIC
'75 LeetCode Questions.gsheet'
 8515E47F-3516-4762-95D9-02E17EB44C8E.JPG
'A1 project backup.gdoc'
'about me RA.gdoc'
'ANTH454 Voyages Project.gslides'
'Bullentin Headlines.gdoc'
'Colab Notebooks'
'Common 301 issues.gdoc'
 COMP283
'COMP301 Course Assessment.gform'
'COMP301 M2 Practice.gdoc'
'COMP311 Exam Review.gdoc'
'Copy of 605  For

In [3]:
# Install Huggingface Transformers library
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00


In [4]:
# Import necessary libraries
import os
from os.path import join
import numpy as np
import pandas as pd
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

In [5]:
train_data = pd.read_csv("/content/drive/My Drive/Corona_NLP_train.csv",encoding='latin-1')
test_data = pd.read_csv("/content/drive/My Drive/Corona_NLP_test.csv",encoding='latin-1')

In [6]:
# set the seed (very important)
seed = 35

In [7]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(seed)

In [8]:
for row in range(len(train_data)):
    if train_data.at[row,"Sentiment"] == "Extremely Negative" or train_data.at[row,"Sentiment"] == "Negative":
        train_data.at[row,"Sentiment"] = 0
    elif train_data.at[row,"Sentiment"] == "Extremely Positive" or train_data.at[row,"Sentiment"] == "Positive":
        train_data.at[row,"Sentiment"] = 1
    elif train_data.at[row,"Sentiment"] == "Neutral":
        train_data = train_data.drop(row)
        
for row in range(len(test_data)):
    if test_data.at[row,"Sentiment"] == "Extremely Negative" or test_data.at[row,"Sentiment"] == "Negative":
        test_data.at[row,"Sentiment"] = 0
    elif test_data.at[row,"Sentiment"] == "Extremely Positive" or test_data.at[row,"Sentiment"] == "Positive":
        test_data.at[row,"Sentiment"] = 1
    elif test_data.at[row,"Sentiment"] == "Neutral":
        test_data = test_data.drop(row)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
# Your Code Here
# 1. Getting train_text, test_text, (i.e. the training and the test tweets.)
#    make sure train_text and test_text is a list of strs.
#    Again, we emphasize that you do NOT have to do any NTLK-typed
#    pre-processing here, just getting raw tweets is sufficient. 
train_text = [tweet for tweet in train_data["OriginalTweet"].tolist()]
test_text = [tweet for tweet in test_data["OriginalTweet"].tolist()]
# 2. Getting train_label, test_label with each being a numpy array and each entry 
#    is 1 for "Positive" and 0 for "Negative"
#    (Remeber that you have to turn the "Extremely Positive" and "Extremely Negative"
#    label to "Positive" and "Negative" and get rid of "Neutral" tweets like you 
#    did in the previous question)
train_label = [tweet for tweet in train_data["Sentiment"].tolist()]
test_label = [tweet for tweet in test_data["Sentiment"].tolist()]

In [9]:
# download pretrained BERT Tokenizer
from transformers import BertTokenizer
# credits to https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python
model_checkpoint = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

In [10]:
# download pretrained BERT model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Your Code Here
# Filling the number of labels in this classification task.
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
# Information of BERT model
def get_model_layer(model):
  return model.config.num_hidden_layers

def get_hidden_size(model):
  return model.config.hidden_size

In [12]:
# Move the model to GPU
# Revisit the GPU tutorial if you meets error in this cell
model = model.to('cuda')

In [13]:
# Metrics

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
  return {
      'accuracy': acc,
      "tn": tn,
      "fp": fp,
      "fn": fn,
      "tp": tp
  }

In [14]:
# Change the batch size here
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    save_steps=200,
    evaluation_strategy="steps",     # evaluate each `logging_steps`?
)


In [15]:
# Encode the data

max_length = 128
train_encodings = tokenizer(train_text, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=max_length)

In [16]:
# Build the dataset

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = TweetDataset(train_encodings, train_label)
test_dataset = TweetDataset(test_encodings, test_label)

In [17]:
# Setup the trainer

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [18]:
# train the model
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33444
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1569
  Number of trainable parameters = 108311810


Step,Training Loss,Validation Loss,Accuracy,Tn,Fp,Fn,Tp
200,0.586300,0.436330,0.804026,1424,209,414,1132
400,0.346900,0.337287,0.853413,1453,180,286,1260
600,0.252600,0.269168,0.884869,1507,126,240,1306
800,0.189600,0.245615,0.904058,1470,163,142,1404
1000,0.172000,0.224502,0.911922,1441,192,88,1458
1200,0.092700,0.295935,0.906889,1431,202,94,1452
1400,0.065400,0.242800,0.921044,1516,117,134,1412


***** Running Evaluation *****
  Num examples = 3179
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3179
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3179
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-600
Configuration saved in ./results/checkpoint-600/config.json
Model weights saved in ./results/checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3179
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
****

TrainOutput(global_step=1569, training_loss=0.22405738538811357, metrics={'train_runtime': 2184.7398, 'train_samples_per_second': 45.924, 'train_steps_per_second': 0.718, 'total_flos': 6599614601594880.0, 'train_loss': 0.22405738538811357, 'epoch': 3.0})

In [19]:
# evaluate the model
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3179
  Batch size = 64


{'eval_loss': 0.22450163960456848,
 'eval_accuracy': 0.9119219880465556,
 'eval_tn': 1441,
 'eval_fp': 192,
 'eval_fn': 88,
 'eval_tp': 1458,
 'eval_runtime': 22.7038,
 'eval_samples_per_second': 140.021,
 'eval_steps_per_second': 2.202,
 'epoch': 3.0}

In [20]:
def get_prediction(model, text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs

In [23]:
print(get_model_layer(model))

12


In [25]:
print(get_hidden_size(model))

768


In [27]:
tweet = 'COVID is already old news. I’m ready to go back to the clubs now #reborn.'
get_prediction(model,tweet)

tensor([[0.0015, 0.9985]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [36]:
print(len(test_label))
print("The number of positive tweets is: " + str(sum(test_label)))
print("The number of negative tweets is: " + str(len(test_label) - sum(test_label)))
print("Predicting all negative tweets would make us right: " + str(1633/3179) + " of the time")

3179
The number of positive tweets is: 1546
The number of negative tweets is: 1633
Predicting all negative tweets would make us right: 0.5136835482856245 of the time


In [40]:
# F score is given by TP / TP + 1/2(FP + FN)
num = 1633
denom = 1633 + (0.5*(1546+0))
print("F score is: " + str(num / denom))

F score is: 0.6787198669991688
